In [38]:
%load_ext autoreload
%autoreload 2

import numpy as np
import numba as nb
import scipy as sp
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
from pathlib import Path
import vrExperiment as vre
import vrFunctions as vrf
import basicFunctions as bf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Steps to code: 
# 1. consider replacing some of the functions in "multiSessionAnalysis"...
# 2. add red cell processing functions into the pipeline

# immediate goals: 
# 1. write functions for retrieving useful variables from oneData formats (e.g. speed, occupancy maps, etc.)
# 2. do I save these? how long will it take to load? 

# BIG GOAL:
# 1. Database time! Do this then rerun all your sessions
# 2. Create smart loading system where each vrexp object loads one data into a floating dictionary, and every time something is needed it checks if it's loaded it and then loads it only when necessary
#    For this to work, I'll have to exclusively use functions that are called on each vrexp object... it's a little tricky but I think it can work

# -- before databasing, I want to test transformed data performance on a few sessions, i.e. I want to see how quickly I can regenerate key behavioral and neural variables from the onedata 

In [39]:
# vrExperiment registration and preprocessing 
mouseName = 'ATL022'
dateString = '2023-03-24'
session = '701'
vrReg = vre.vrExperimentRegistration(mouseName, dateString, session, oasis=False, imaging=True)
vrReg.doPreprocessing()
vrReg.saveParams()
print(vrReg.sessionPath())

In session: ATL022/2023-03-24/701, vrFile['rigInfo'] does not exist. Assuming default settings for B2!


C:\Users\andrew\anaconda3\envs\vrAnalysis\lib\site-packages\scipy\stats\_stats_py.py:2855: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  a = np.asanyarray(compare)


TypeError: unsupported operand type(s) for +: 'float' and 'slice'

In [162]:
# Post-processing summary variables
# 1. convolve spks ==========================Note: I don't think I'm going to keep doing this. It barely changes the spike vector and we're already using GCaMP data anyway...
# 2. occupancy map, speed map, lick map ================ finished
# 3. spkmap for position (with spatial smoothing) =================== finished (and I wrote a toeplitz convolution that's fast)
# 4. reliability measurements and spatial information measurements 
# 5. ROICaT alignment tools 

# create all these variables (make sure you did it correctly), and time how long it takes to load them
# then save them all and time it takes how long to load them and how much data to save them

In [4]:
mouseName = 'ATL022'
dateString = '2023-03-24'
session = '701'
vrexp = vre.vrExperiment(mouseName, dateString, session)

In [19]:
%%time
# Retrieve smoothed and downsampled occupancymap and speedmap
omap, smap, distvec = vrexp.getBehaviorMaps(speedThreshold=5) # produce smoothed occupancy and speed maps, along with the distance bins used to create them
frameTrialIdx, framePosition, frameSpeed = vrexp.getFrameBehavior()
spkmap = vrexp.getSpikeMap(frameTrialIdx, framePosition, frameSpeed, distvec, standardizeSpks=True, doSmoothing=5) # produce the spkmap 

Lick map should be included in 'getBehaviorMaps' eventually!
CPU times: total: 13.5 s
Wall time: 3.95 s


In [26]:
%time mse, cor = measureReliability(spkmap, numcv=3)

CPU times: total: 359 ms
Wall time: 350 ms


In [38]:
# Next step: 
# integrate spatial reliability code into vrExperiment
# Write code to measure spatial information

In [28]:

    
def vectorCorrelation(x,y):
    # for each column in x, measure the correlation with each column in y
    assert x.shape==y.shape, "x and y need to have the same shape!"
    N = x.shape[0]
    xDev = x - np.mean(x,axis=0)
    yDev = y - np.mean(y,axis=0)
    xSampleStd = np.sqrt(np.sum(xDev**2,axis=0)/(N-1))
    ySampleStd = np.sqrt(np.sum(yDev**2,axis=0)/(N-1))
    xStandard = xDev/xSampleStd
    yStandard = yDev/ySampleStd
    return np.sum(xStandard * yStandard,axis=0) / (N-1) 

def cvFoldSplit(numSamples, numFold):
    minimumSamples = np.floor(numSamples / numFold)
    remainder = numSamples - numFold*minimumSamples
    samplesPerFold = [int(minimumSamples + 1*(f<remainder)) for f in range(numFold)]
    sampleIdxPerFold = [0, *np.cumsum(samplesPerFold)]
    randomOrder = np.random.permutation(numSamples)
    foldIdx = [randomOrder[sampleIdxPerFold[i]:sampleIdxPerFold[i+1]] for i in range(numFold)]
    return foldIdx